In [15]:
# ------------------------------------------------------
# 02_FEATURE_ENGINEERING.ipynb
# Complete KPI + Business + Creative Feature Engineering
# ------------------------------------------------------

import pandas as pd
import numpy as np

DATA_PATH = "../data/raw/campaign_performance.csv"

df = pd.read_csv(DATA_PATH)
df.head()

# Replace blanks with NaN
df = df.replace("", np.nan)

# Critical columns required for KPIs and ML
critical_cols = [
    'timestamp','campaign_id','ad_id','impressions','clicks',
    'conversions','spend','revenue'
]

df = df.dropna(subset=critical_cols).reset_index(drop=True)
print("Rows after cleaning:", len(df))

def add_time_features(df, date_col="timestamp"):
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    df['hour_of_day'] = df[date_col].dt.hour
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['week_of_year'] = df[date_col].dt.isocalendar().week.astype(int)
    df['is_weekend'] = df['day_of_week'] >= 5

    # Cyclic encodings for ML
    df['hour_sin'] = np.sin(2*np.pi * df['hour_of_day']/24)
    df['hour_cos'] = np.cos(2*np.pi * df['hour_of_day']/24)

    return df

df = add_time_features(df)
df[['hour_of_day','day_of_week','is_weekend']].head()

def add_kpi_features(df):
    df['CTR'] = df['clicks'] / df['impressions'].replace(0, np.nan)
    df['CVR'] = df['conversions'] / df['clicks'].replace(0, np.nan)
    df['CPC'] = df['spend'] / df['clicks'].replace(0, np.nan)
    df['CPM'] = (df['spend'] / df['impressions'].replace(0, np.nan)) * 1000
    df['CPA'] = df['spend'] / df['conversions'].replace(0, np.nan)
    df['ROAS'] = df['revenue'] / df['spend'].replace(0, np.nan)
    df['RPC'] = df['revenue'] / df['clicks'].replace(0, np.nan)  # revenue per click
    return df

df = add_kpi_features(df)
df[['CTR','CVR','ROAS','CPA','CPC']].head()

def add_business_features(df):
    if 'daily_budget_campaign' in df.columns:
        df['budget_utilization'] = df['spend'] / df['daily_budget_campaign'].replace(0,np.nan)

    df['impressions_per_click'] = df['impressions'] / df['clicks'].replace(0,np.nan)

    df['performance_index'] = (
        0.5 * df['CTR'] +
        0.3 * df['CVR'] +
        0.2 * df['ROAS']
    )
    return df

df = add_business_features(df)
df[['budget_utilization','impressions_per_click','performance_index']].head()

def add_rolling_features(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    df = df.sort_values(["campaign_id", "timestamp"])

    # Rolling metrics applied on FULL group, not a Series
    df['roll_spend_1h'] = (
        df.groupby("campaign_id")
          .apply(lambda g: g.rolling("1h", on="timestamp")['spend'].sum())
          .reset_index(level=0, drop=True)
    )

    df['roll_clicks_1h'] = (
        df.groupby("campaign_id")
          .apply(lambda g: g.rolling("1h", on="timestamp")['clicks'].sum())
          .reset_index(level=0, drop=True)
    )

    df['roll_conv_1h'] = (
        df.groupby("campaign_id")
          .apply(lambda g: g.rolling("1h", on="timestamp")['conversions'].sum())
          .reset_index(level=0, drop=True)
    )

    df['roll_ROAS_24h'] = (
        df.groupby("campaign_id")
          .apply(lambda g: g.rolling("24h", on="timestamp")['ROAS'].mean())
          .reset_index(level=0, drop=True)
    )

    df['roll_CTR_24h'] = (
        df.groupby("campaign_id")
          .apply(lambda g: g.rolling("24h", on="timestamp")['CTR'].mean())
          .reset_index(level=0, drop=True)
    )

    return df
df = add_rolling_features(df)
df[['roll_spend_1h','roll_CTR_24h','roll_ROAS_24h']].head()


def creative_intelligence(df):
    creative = df.groupby('ad_id').agg({
        'CTR':'mean',
        'CVR':'mean',
        'ROAS':'mean',
        'spend':'sum',
        'revenue':'sum'
    }).reset_index()

    creative['creative_ROAS'] = creative['revenue'] / creative['spend'].replace(0,np.nan)
    creative['creative_score'] = 0.5*creative['CTR'] + 0.5*creative['CVR']

    return creative

creative_features = creative_intelligence(df)
creative_features.head()


def audience_geo_features(df):
    audience = df.groupby("audience_segment").agg({
        'CTR':'mean','CVR':'mean','spend':'sum','revenue':'sum'
    }).reset_index()
    audience['audience_ROAS'] = audience['revenue'] / audience['spend']

    geo = df.groupby("country").agg({
        'CTR':'mean','CVR':'mean','spend':'sum','revenue':'sum'
    }).reset_index()
    geo['geo_ROAS'] = geo['revenue'] / geo['spend']

    return audience, geo

audience_features, geo_features = audience_geo_features(df)
audience_features.head()


def online_snapshot(df):
    snap = df.groupby('campaign_id').agg({
        "CTR":"mean",
        "CVR":"mean",
        "ROAS":"mean",
        "CPA":"mean",
        "performance_index":"mean",
        "spend":"sum",
        "impressions":"sum",
        "clicks":"sum",
        "conversions":"sum"
    }).reset_index()

    snap["quality_signal"] = (
        0.4*snap["CTR"] +
        0.3*snap["CVR"] +
        0.3*snap["ROAS"]
    )

    return snap

snapshot = online_snapshot(df)
snapshot.head()

final_features = df.copy()
final_features.head()



Rows after cleaning: 60000


C:\Users\VIR MACHRA\AppData\Local\Temp\ipykernel_49896\4077801409.py:81: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.rolling("1h", on="timestamp")['spend'].sum())
C:\Users\VIR MACHRA\AppData\Local\Temp\ipykernel_49896\4077801409.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.rolling("1h", on="timestamp")['clicks'].sum())
C:\Users\VIR MACHRA\AppData\Local\Temp\ip

,event_id,timestamp,channel,campaign_id,ad_group_id,ad_id,country,industry,device_type,placement,...,ROAS,RPC,budget_utilization,impressions_per_click,performance_index,roll_spend_1h,roll_clicks_1h,roll_conv_1h,roll_ROAS_24h,roll_CTR_24h
5698,sdv-id-zYbhFh,2025-09-18 07:03:03,Video,CMP_001,AG_036,AD_0353,BR,Tech,Desktop,Search,...,0.0,0.0,0.004800,15.076923,0.033163,7.98,13.0,0.0,0.0,0.066327
8284,sdv-id-CsMark,2025-09-18 07:03:03,Social,CMP_001,AG_094,AD_0284,DE,Tech,Desktop,Search,...,0.0,0.0,0.015440,23.300000,0.021459,36.02,33.0,0.0,0.0,0.054622
9975,sdv-id-eGkMfi,2025-09-18 07:03:03,Video,CMP_001,AG_034,AD_0231,DE,Retail,Mobile,In-stream,...,0.0,NaN,0.012992,NaN,NaN,57.24,33.0,0.0,0.0,0.036415
15165,sdv-id-qQgEnb,2025-09-18 07:03:03,Search,CMP_001,AG_060,AD_0079,BR,Gaming,Desktop,Banner,...,0.0,0.0,0.042943,18.500000,0.027027,87.78,41.0,0.0,0.0,0.040825
18205,sdv-id-lJJKwl,2025-09-18 07:03:03,Search,CMP_001,AG_083,AD_0548,US,Auto,Desktop,Feed,...,0.0,0.0,0.002043,24.520000,0.020392,95.59,66.0,0.0,0.0,0.040816


In [18]:

# FIX: Convert 'is_weekend' boolean → numeric (0/1)


if "is_weekend" in final_features.columns:
    final_features["is_weekend"] = final_features["is_weekend"].astype(int)

print(final_features["is_weekend"].value_counts())


is_weekend
0    43390
1    16610
Name: count, dtype: int64


In [20]:

import os

# Create folder if it doesn't exist
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Save the final engineered dataset
output_path = os.path.join(output_dir, "features1.csv")
final_features.to_csv(output_path, index=False)

print(f"Feature dataset saved successfully at: {output_path}")


Feature dataset saved successfully at: ../data/processed\features1.csv
